**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras almacenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
#configuración del laboratorio
# Ejecuta esta celda!
%load_ext autoreload
%autoreload 2
in_colab = True

In [ ]:
import os
if not in_colab:
    import sys ; sys.path.append('../commons/utils/'); sys.path.append('../commons/utils/data')
else: 
    os.system('wget https://raw.githubusercontent.com/mariabda2/ML_2022/master/Labs/commons/utils/general.py -O general.py')
    from general import configure_lab5_2
    configure_lab5_2()
from lab5 import *
GRADER, dataset = part_2()
import seaborn as sns

# Laboratorio 5 - Parte 2. Máquinas de Vectores de Soporte

## Ejercicio 1: Limipiar base de datos y completar código

En este ejercicio usaremos la regresión por vectores de soporte para resolver el problema de regresión de la base de datos AirQuality (https://archive.ics.uci.edu/ml/datasets/Air+Quality). Tener en cuenta que vamos a usar solo 2000 muestras.

En primera instancia vamos a transformar la matriz en un dataframe, para poderlo procesar de manera mas sencilla. Se crea una columna por cada variable que obtenemos.

In [ ]:
dataset_df = pd.DataFrame(dataset, columns = [f'col_{c}' for c in range (1,14)])
dataset_df

Para esta base de datos vamos a realizar una limpieza de datos. 

Para ello vamos a completar la siguiente función:
    
1. **Remover** todos registros cuya variable objetivo es faltante (missing Value). Estos registros están marcados como -200, es decir, donde haya un valor -200 eliminaremos el registro.
2. **imputar los valores perdidos/faltantes** en cada una de las características, vamos asuar la mediana de la característica en especifico.
3. **Verificar** si quedaron valores faltantes
4. **retornar**: X (12 primeras columnas) y Y(la 13 columna).
Tips:
1. Nuestra [sesión extra](https://mariabda2.github.io/ML_2022/Labs/Extra/Basic_Preprocessing_FeatureEngineering.html).
2. Para transformar columnas de pandas a arreglos de numpy se puede usar `.iloc` / `.loc` y . `.values`, por ejemplo  para devolver una matriz con los valores de las primeras dos columnas es posible hacerlo asi: `dataset_df.iloc[: , 0:2].values` o `dataset_df.loc[: , ['col_1', 'col_2']].values`.

In [ ]:
# ejercicio de codigo
def clean_data(df):
    """Función que limpia el dataset y obtiene X y Y.
    
    Argumentos:

        df: es un pandas dataframe.
    
    Retorna:
        X: una matriz numpy con los valores a usar como conjunto de datos
        de entrada.
        Y una matriz numpy con los valores usados como conjunto de datos
        de salida.
    
    """
    
    # se copia el df para evitar cambios sobre el objeto
    database = df.copy()

    # Reemplazar por el valor correcto
    VALOR_FALTANTE = ...
    
    ## Completar con la operación adecuada

    pct_valores_faltantes = (database==-VALOR_FALTANTE)...()
    # imprimir resumen
    print(",".join([f"% VF en {a}: {pct_valores_faltantes[a]*100:.2f}% " for a in pct_valores_faltantes.index]))
    
    # identificar muestras cuya salida en un valor faltante

    idx_to_remove = []
    for idx in database.index:
        ## reemplazar el valor adecuado
        if database.iloc[idx, ...] == VALOR_FALTANTE:
            idx_to_remove.append(idx)
    
    #remover la muestras de los indices
    database = database.drop(idx_to_remove, axis = 0)

    print ("\nHay " + str(len(idx_to_remove)) + " valores perdidos en la variable de salida.")
    print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(database)))

    ## Imputar usando la estrategia adecuada.
    print ("\nProcesando imputación de valores perdidos en las características . . .")
    imputer = ...(missing_values= ... , ...)
    "imputar solo las columnas de las variables de entrada"
    database.iloc[:, ... : ...] = imputer.fit_transform(database.iloc[:, ...] )

    print ("Imputación finalizada.\n")

    ## Completar con la operación adecuada
    pct_valores_faltantes = (database==...)...()
    
    print(",".join([f"% VF en {a}: {pct_valores_faltantes[a]*100:.2f}% " for a in pct_valores_faltantes.index]))
    
    if(pct_valores_faltantes.max() != 0):
        print ("Hay valores perdidos")
    else:
        print ("No hay valores perdidos en la base de datos. Ahora se puede procesar")

    X = database.iloc[:, ... : ...].values
    Y = database.iloc[:, ... : ...].values
    return (X,Y)

In [ ]:
# Ignorar los prints.
GRADER.run_test("ejercicio1", clean_data)

Ahora usemos la función para tener nuestras variables X, Y

In [ ]:
X,Y = clean_data(dataset_df)

## Ejercicio 2: Experimentar SVM para regresión
 
Ahora vamos a crear la función para experimentar con la máquina de soporte vectorial con las siguientes especificaciones:
 
1. Usar la librería de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html.
2. Variar tres parámetros del SVR: kernel,  gamma y el parámetro de regularización.
3. Utilizar la metodología de validación cruzada con 5 folds.
4. Usar normalización de datos estándar implementada por sklearn
5. Extraer los vectores de soporte (observe los *atributos* del modelo SVR de sklearn). Recuerde que estos atributos pueden ser accedidos, una vez el modelo es entrenado.
6. Utilizar la métrica para calcular el MAPE (usar sklearn).
 
**Notas**:
- Deberíamos poder acceder a las funciones de la librería de sklearn directamente por el nombre sin necesidad de importarlas. Las funciones que deberías utilizar ya están precargadas en la configuración del laboratorio.

- Llame todos los parámetros de las funciones de sklearn de manera explícita. (i.e, si se quiere usar `max_iter` como parámetro para el SVR, debe crear el objeto: `SVR(max_iter = 100)`)

In [ ]:
#ejercicio de código
def experiementarSVR(x, y, kernels, gammas,params_reg):
    """función que realizar experimentos sobre un SVM para regresión
    
    Argumentos
        x: numpy.Array, con las caracteristicas del problema
        y: numpy.Array, con la variable objetivo
        kernels: List[str], lista con valores a pasar 
            a sklearn correspondiente al kernel de la SVM
        gammas: List[float], lista con los valores a pasar a
            sklean correspondiente el valor de los coeficientes para usar en el
            kernel
        params_reg: List[float], lista con los valores a a pasar a 
            sklearn para ser usados como parametro de regularización
    retorna: 
        pd.Dataframe con los resultados.
    """
    idx = 0
    kf = KFold(n_splits= ... )
    # crear una lista con la combinaciones de los elementos de cada lista.
    kernels_gammas_regs = list(itertools.product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) # puede usar para ver los params
        rendimiento_test = []
        pct_support_vectors = []
        num_support_vectors = []
        for train_index, test_index in kf.split(x):
            
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  
            # normalizar los datos
            scaler = ... ()
            X_train = scaler... (X=...)
            X_test = scaler.... (X=...)
            svm = ... ( )
            # Entrenar el modelo
            svm... (X=... , y=...)
            # Validación del modelo
            ypred = svm.... (X=...)
            
            # error y pct de vectores de soporte
            rendimiento_test.append(... (...  = .... , ... = ...))
            # contar muestras de entrenamiento
            n_train = ... 
            num_vs = len(svm...)
            pct_vs = (num_vs/ n_train ) *100
            pct_support_vectors.append(pct_vs)
            num_support_vectors.append(num_vs)
        
        resultados.loc[idx,'kernel'] = ...
        resultados.loc[idx,'gamma'] = ...
        resultados.loc[idx,'param_reg'] = ... 
        resultados.loc[idx,'Métrica de de rendimiento'] = np... (rendimiento_test)
        resultados.loc[idx,'Desviación estandar en métrica de de rendimiento'] = np... (rendimiento_test)
        resultados.loc[idx,'# de vectores de soporte'] = np.mean(...)
        resultados.loc[idx,'% de vectores de soporte'] = np.mean(...)
        
        idx+=1
    return (resultados)

In [ ]:
GRADER.run_test("ejercicio2", experiementarSVR)

Para entrenar vamos a ignorar las dos primeras variables, estas corresponden a valores de fechas.

In [ ]:
# vamos a realizar los experimentos.
resultadosSVR = experiementarSVR(x =X[:,2:],y=Y,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.1, 1.0,10]
                                )

In [ ]:
# ver los 5 primeros resultados
resultadosSVR.sort_values('Métrica de de rendimiento',ascending=True).head(3)

In [ ]:
# Ver los 5 peores modelos
# ver los 5 primeros resultados
resultadosSVR.sort_values('Métrica de de rendimiento',ascending=False).head(3)

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Cual es el objetivo de las las funciones kernel? Contestar dentro del contexto de las máquinas de sporte vectorial.
respuesta_1 = "" #@param {type:"string"}

In [ ]:
#@title Pregunta Abierta
#@markdown Explique en sus palabras ¿cual es la relación tienen los vectores de soporte y el epsilon-tubo?
respuesta_2 = "" #@param {type:"string"}

Para analizar los resultados comparando el mejor y el peor modelo encontrado, usando dos graficas:

1. Grafica donde el eje x es el valor real y eje y el valor predicho.
2. Grafica donde el eje x vamos a dejar un valor incremental y con colores vamos a diferenciar entre el valor real y el valor predicho

In [ ]:
# dividir el conjunto para estas graficas
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#predicciones
# OJO: Reemplazar los valores!
YpredBest =  predict_svr(X_train, y_train, X_test, y_test, kernel = ... , gamma = ... , param_reg = ...)
YpredWorst = predict_svr(X_train, y_train, X_test, y_test, kernel = ... , gamma = ... , param_reg = ... )
# verificar en el print que el error sea diferente y concuerde con el entendimiento.

In [ ]:
# plots
f, ax = plt.subplots(nrows=1, sharex=False, sharey=False, figsize = (16,8))
# Mejor Modelo.
ax.scatter(y_test, YpredBest, label = 'Mejor Modelo')
ax.scatter(y_test, YpredWorst, label = 'Peor Modelo', c='r')
ax.plot(y_test, y_test, label = 'Modelo perfecto', c='k', alpha=0.5)
ax.set_xlabel('valor real', fontdict = {'fontsize': 12})
ax.set_ylabel('valor predicho', fontdict = {'fontsize': 12})
ax.set_title("Mejor modelo vs Peor modelo")
ax.legend()

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Analizando de manera cualitativa la gráfica?
#@markdown ¿Cuál de las afirmaciones está mejor justificada?
 
#@markdown A) En los dos modelos se observa una relación lineal, sin embargo el "peor" modelo tiene mayores diferencias en los valores extremos (cercanos a 0.4 y mayores que 1.4).
 
#@markdown B) En los dos modelos se observa una relación lineal, sin embargo el "mejor" modelo tiene mayores diferencias al observar los valores extremos (cercanos a 0.4 y mayores que 1.4).
 
#@markdown C) En los dos modelos se observa una relación lineal,  no es posible determinar diferencias ya que se esperaba que "peor" modelo no tuviera una relación lineal.
 
#@markdown D) Se observa que en los valores centrales, la diferencia no es mucha, las diferencias se ven en los valores extremos que pueden ser catalogados como "outliers", por lo tanto no hay una diferencia significativa en los dos modelos.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_3 = '' #@param ["", "A", "B", "C", "D"]


In [ ]:
f, ax = plt.subplots(nrows=2, sharex=False, sharey=False, figsize = (16,8))
#  Mejor Modelo
ax[0].plot(y_test, label = 'valor real', color = 'b', alpha = 0.7)
ax[0].plot(YpredBest, label = 'valor predicho mejor modelo', color = 'r',linestyle='dashed', alpha = 0.5)
ax[0].legend()
ax[0].set_ylabel('Humedad relativa', fontdict = {'fontsize': 12})
#  Peor Modelo
ax[1].plot(y_test, label = 'valor real', color = 'b', alpha = 0.7)
ax[1].plot(YpredWorst, label = 'valor predicho peor modelo', color = 'r',linestyle='dashed', alpha = 0.5)
ax[1].legend()
ax[1].set_ylabel('Humedad relativa', fontdict = {'fontsize': 12})
plt.show()

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Analizando de manera cualitativa las gráficas?
#@markdown ¿Cuál de las afirmaciones está mejor justificada?
 
#@markdown A) En el "peor" modelo se observa menores diferencias, en ciertas partes de la gráfica se puede diferenciar las dos líneas de manera muy clara. En el "mejor" modelo también ocurren estas diferencias, pero los valores están más alejados.
 
#@markdown B) Ningún modelo puede resolver bien el problema, debemos seguir entrenando hasta lograr que los valores predichos no se logren diferenciar del valor real.
 
#@markdown C) En el "mejor" modelo se observa mayores diferencias, en ciertas partes de la gráfica se puede diferenciar las dos líneas de manera muy clara. En el "peor" modelo también ocurren estas diferencias, pero los valores son más cercanos.
 
#@markdown D) En el "peor" modelo se observa mayores diferencias, en ciertas partes de la gráfica se puede diferenciar las dos líneas de manera muy clara. En el "mejor" modelo también ocurren estas diferencias, pero los valores son más cercanos.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_4 = '' #@param ["", "A", "B", "C", "D"]

## Ejercicio 3: Experimentar SVM para clasificación

En este ejercicio vamos a volver a resolver el problema de clasificación de dígitos. Vamos usar solo 5 clases y realizaremos un pre-procesamiento:
1. mediante PCA (una tecnica proxima a practicar en el laboratorio)
2. Vamos a convertirlo en problema de tres clases (vamos a diferenciar entre 0, 1 y el resto)

In [ ]:
Xcl, Ycl = load_digits(n_class=5,return_X_y=True)
#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
Xcl = pca.fit_transform(Xcl)
# cambiar problema de clases
unique, counts  = np.unique(Ycl, return_counts=True)
print("distribución original (claves las etiquetas, valores el número de muestras): \n", dict(zip(unique, counts )))
Ycl = np.where(np.isin(Ycl, [0,1]), Ycl, 2)
unique, counts  = np.unique(Ycl, return_counts=True)
print("Nueva distribución  (claves las etiquetas, valores el número de muestras): \n", dict(zip(unique, counts )))

Ahora vamos a crear la función para experimentar con la maquina de soporte vectorial para un problema de clasificación. Para ello vamos a:

1. Usar la libreria de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
2. Variar tres parámetros del SVC: kernel,  gamma y el parametro de regularización.
3. Utilizar la metodología cross-validation con 4 folds más adecuada para problemas de clasificación.
4. Usar la normalización de datos estandar implementada por sklearn.
5. Extraer los vectores de soporte (observe los *atributos* del modelo SVC de sklearn). Recuerde que estos atributos son accesibles una vez el modelo es entrenado
6. vamos a probar la dos estragegias del SVC:
    - One Vs One
    - One Vs Rest
7. Utilizar como error el score de exactitud de la clasificación de sklearn.

**Notas**: 
- Deberiamos poder acceder a las funciones de la libreria de sklearn directamente por el nombre sin necesidad de importarlas. Las funciones que deberios utilizar ya están precargadas en la configuración del laboratorio.
- Llame todos los parametros de las funciones de sklearn de manera explicita. (i.e, si se quiere usar `max_iter` como parámetro para el SVC, debe crear el objeto: `SVC(max_iter = 100)`)

* sklearn tiene unos "wrappers", que implementan estrategias para la clasificación multiclase, uno  de estos wrappers, implementa la estrategia one-vs-rest: https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html.

* Un wrapper, es un esquema de diseño común para "envolver" librerias/funciones con caracteristicas similares y poder modificar ciertos comportamientos.

In [ ]:
#ejercicio de código
def experiementarSVC(x, y, kernels, gammas,params_reg, estrategia = 'ovo'):
    """función que realizar experimentos sobre un SVM para clasificación
    
    x: numpy.Array, con las caracteristicas del problema
    y: numpy.Array, con la variable objetivo
    kernels: List[str], lista con valores a pasar 
        a sklearn correspondiente al kernel de la SVM
    gammas: List[float], lista con los valores a pasar a
        sklean correspondiente el valor de los coeficientes para usar en el
        kernel
    params_reg: List[float], lista con los valores a a pasar a 
        sklearn para ser usados como parametro de regularización
    estrategia: str, valor que puede ser ovo (para one vs one) o ovr 
        (para one vs rest)
    
    retorna: 
        pd.Dataframe con el resultado de los experimentos.
    """
    idx = 0
    kf = ... (n_splits= ... )
    # crear una lista con la combinaciones de los elementos de cada list
    kernels_gammas_regs = list(itertools.product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) # puede usar para ver los params
        rendimiento_train = []
        rendimiento_test = []
        pct_support_vectors = []
        
        for train_index, test_index in kf.split(X = ... , y = ... ):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  
            # normalizar los datos
            scaler = ... ()
            X_train = scaler.... (X = X_train)
            X_test = scaler... (X = X_test)
            
            svm = ... (...)
            
            # si es estrategia "envolver" a la svm
            if estrategia =='ovr':
                svm = ... (svm)               
            
            # Entrenar el modelo
            svm.fit(X=X_train, y=y_train)
            # calculo de errores
            y_train_pred = svm... (X=... )
            y_test_pred = svm... (X=...)
            # error y pct de vectores de soporte
            rendimiento_train.append(... (y_true = y_train, y_pred = y_train_pred))
            rendimiento_test.append(... (y_true = y_test, y_pred = y_test_pred))
            # contar muestras de entrenamiento
            n_train = ...
            if estrategia == 'ovr':
                # en esta estrategia se realizar una SVM por cada clase
                # por lo tanto tenemos que acceder a cada una de la

                num_vs = np.mean([len(svc...) for svc in svm.estimators_])
                pct_vs = (num_vs/n_train)*100
                
            else:
                pct_vs = (len(svm.support_)/n_train)*100
            pct_support_vectors.append(pct_vs)
        
        resultados.loc[idx,'kernel'] = ...
        resultados.loc[idx,'gamma'] = ...
        resultados.loc[idx,'param_reg'] = ...
        resultados.loc[idx,'estrategia'] = ... 
        resultados.loc[idx,'Métrica de de rendimiento entrenamiento'] = np... (rendimiento_train)
        resultados.loc[idx,'Métrica de de rendimiento prueba'] = np... (rendimiento_test)
        resultados.loc[idx,'% de vectores de soporte'] = np... (pct_support_vectors)
        idx+=1
    return (resultados)

In [ ]:
GRADER.run_test("ejercicio3", experiementarSVC)

Veamos la estrategia OVR

In [ ]:
# vamos a realizar los experimentos
resultadosSVC_ovr = experiementarSVC(x = Xcl,y=Ycl,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.001, 0.01,0.1, 1.0,10],
                                estrategia = 'ovr')

In [ ]:
# ver los mejores modelos
resultadosSVC_ovr.sort_values('Métrica de de rendimiento prueba', ascending=False).head(3)

Ahora vamos a ver la estrategia OVO

In [ ]:
# vamos a realizar los experimentos
resultadosSVC_ovo = experiementarSVC(x = Xcl,y=Ycl,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.001, 0.01,0.1, 1.0,10],
                                estrategia = 'ovo')

In [ ]:
# ver los mejores modelos
resultadosSVC_ovo.sort_values('Métrica de de rendimiento prueba', ascending=False).head(3)

In [ ]:
#@title Pregunta Abierta
#@markdown Explique en sus palabras ¿qué representan los vectores de soporte en un problema de clasificación?
respuesta_5 = "" #@param {type:"string"}

In [ ]:
 
#@title Pregunta Abierta
#@markdown ¿Cual otra métrica del módulo de sklearn podría ser usada?
#@markdown ¿Cuál de las afirmaciones está mejor justificada?
 
#@markdown A) Por ser un problema de clasificación sin importar el balance de clases, el `accuracy_score` es adecuado, pero se podría usar también `f1_score` por ser un problema binario.
 
#@markdown B) Por ser un problema de clasificación desbalanceado  `matthews_corrcoef` y `balanced_accuracy_score` son más adecuadas.
 
#@markdown C) Por ser un problema de clasificación desbalanceado `accuracy_score` es adecuada, pero se podría usar también `f1_score` y `confusion_matrix`.
 
#@markdown D) Por ser un problema de clasificación desbalanceado  `matthews_corrcoef`, `balanced_accuracy_score` y `G_mean` son más adecuadas.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_6 = '' #@param ["", "A", "B", "C", "D"]

In [ ]:
# ver la relación de parametro de regularización y los vectores de soporte
ax = sns.relplot(data = resultadosSVC_ovo, x = 'param_reg', y = '% de vectores de soporte', kind = 'line', col ='kernel', aspect = 1.5)
ax.set(xscale="log")

In [ ]:
#@title Pregunta Abierta
#@markdown ¿qué relación observa entre el valor del parametro de regularización y los vectores de soporte? ¿como puede ser explicada esta relación?
respuesta_7 = "" #@param {type:"string"}

In [ ]:
GRADER.check_tests() 

In [ ]:
#@title Integrantes
codigo_integrante_1 ='' #@param {type:"string"}
codigo_integrante_2 = ''  #@param {type:"string"}

----
esta linea de codigo va fallar, es de uso exclusivo de los profesores


In [ ]:
GRADER.grade()